In [2]:
# !pip install youtube_transcript_api
# !pip install -U -q google.generativeai
# !pip install streamlit
# !pip install python-dotenv
# !pip install Flask
# !pip install flask-cors


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from youtube_transcript_api import YouTubeTranscriptApi
import google.generativeai as genai
from dotenv import load_dotenv
import os

In [5]:
load_dotenv()

True

In [6]:
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [34]:
prompt = """Welcome, Video quiz generator! Your task is to generate a quiz from a given YouTube video transcript, potentially from a video lecture. Your quiz should test on the key points and essential information, presented in 5 multiple choice questions. You should also tell me the answer to each question. You should organize each question and answer into a JSON format, so that there is an array of 5 JSON objects."""

In [9]:
def extract_transcript_details(youtube_video_url):
    try:
        video_id = youtube_video_url.split("=")[1]
        transcript_text = YouTubeTranscriptApi.get_transcript(video_id)

        transcript = ""
        for i in transcript_text:
            transcript += " " + i["text"]

        return transcript
    except Exception as e:
        raise e

def generate_gemini_content(transcript, prompt):
    model = genai.GenerativeModel("gemini-1.5-pro-latest")
    response = model.generate_content(prompt + transcript)
    return response.text


In [35]:
transcript = extract_transcript_details("https://www.youtube.com/watch?v=zO0sSJB1TrI&ab_channel=SRFKulturSternstunden")
response = generate_gemini_content(transcript, prompt)
response

'```json\n[\n  {\n    "question": "In the thought experiment, what happens to the person\'s brain?",\n    "answers": {\n      "a": "It is left inside the body and studied.",\n      "b": "It is removed and placed in a nutrient solution.",\n      "c": "It is scanned and then returned to the body.",\n      "d": "It is destroyed to observe the body\'s reaction."\n    },\n    "correct_answer": "b"\n  },\n  {\n    "question": "Why does the scientist stimulate the brain in the vat?",\n    "answers": {\n      "a": "To control the person\'s thoughts and actions.",\n      "b": "To study the brain\'s response to stimuli.",\n      "c": "To create a realistic sensory experience for the brain.",\n      "d": "To erase the person\'s memories and identity."\n    },\n    "correct_answer": "c"\n  },\n  {\n    "question": "What is the main argument for the brain in a vat scenario being possible?",\n    "answers": {\n      "a": "Our experiences are ultimately just electrical signals in the brain.",\n      

In [36]:
import json
json.loads(response[7:-6])

[{'question': "In the thought experiment, what happens to the person's brain?",
  'answers': {'a': 'It is left inside the body and studied.',
   'b': 'It is removed and placed in a nutrient solution.',
   'c': 'It is scanned and then returned to the body.',
   'd': "It is destroyed to observe the body's reaction."},
  'correct_answer': 'b'},
 {'question': 'Why does the scientist stimulate the brain in the vat?',
  'answers': {'a': "To control the person's thoughts and actions.",
   'b': "To study the brain's response to stimuli.",
   'c': 'To create a realistic sensory experience for the brain.',
   'd': "To erase the person's memories and identity."},
  'correct_answer': 'c'},
 {'question': 'What is the main argument for the brain in a vat scenario being possible?',
  'answers': {'a': 'Our experiences are ultimately just electrical signals in the brain.',
   'b': 'The technology to create such a scenario already exists.',
   'c': 'We have no way of knowing what is real and what is not

In [ ]:
def parse_response(response):
    question_list = []
    choices_list = [[] for _ in range(5)]
    answer_list = []
    
    for i in range(5):
        start = response.find(f"**Question {i+1}:**")
        end = response.find(f"**Question {i+2}:**")